In [2]:
# Import Pandas
import pandas as pd

#Set Relative Path
import sys
sys.path.append("../src")

In [3]:
# Import data
df_dataK = pd.read_csv(r'../data/d03_cleaned_data/datall.csv')
df_dataK.head()

,text,emotion
0,i got up feeling horny this morning,love
1,i will go to my mailbox and talk to the mailma...,happy
2,i understand that any of my extremely positive...,happy
3,i suggest you give it a listen i feel like i a...,love
4,i have been with petronas for years i feel tha...,happy


In [4]:
from nltk.stem import WordNetLemmatizer
def lemmatizer(text):
    """Apply Wordnet lemmatizer to text (go to root word)"""
    wnl = WordNetLemmatizer()
    text = [wnl.lemmatize(word) for word in text.split()]
    return " ".join(text)
df_dataK['text'] = df_dataK['text'].apply(lemmatizer)

In [5]:
# suppression des caratères spéciaux, formatage des contractions...
from nltk.corpus import stopwords
from emoji import demojize
import re

def clean_str(texts):
    from nltk.corpus import stopwords
    # Lowercasing
    texts = texts.str.lower()

    # Remove special chars
    texts = texts.str.replace(r"(http|@)\S+", "")
    texts = texts.apply(demojize)
    texts = texts.str.replace(r"::", ": :")
    texts = texts.str.replace(r"’", "'")
    texts = texts.str.replace(r"[^a-z\':_]", " ")

    # Remove repetitions
    pattern = re.compile(r"(.)\1{2,}", re.DOTALL)
    texts = texts.str.replace(pattern, r"\1")

    # Transform short negation form
    texts = texts.str.replace(r"(can't|cannot)", 'can not')
    texts = texts.str.replace(r"(ain't|wasn't|weren't)", 'be not')
    texts = texts.str.replace(r"(don't|didn't|didnt)", 'do not')
    texts = texts.str.replace(r"(haven't|hasn't)", 'have not')
    texts = texts.str.replace(r"(won't)", 'will not')
    texts = texts.str.replace(r"(im)", ' i am')
    texts = texts.str.replace(r"(ive)", ' i have')
    texts = texts.str.replace(r"(n't)", ' not')

    # Remove stop words
    stopwords = stopwords.words('english')
    stopwords.remove('not')
    stopwords.remove('nor')
    stopwords.remove('no')
    texts = texts.apply(lambda x: ' '.join([word for word in x.split() if (word not in stopwords and len(word) > 1 )]))
    return texts

df_dataK['text'] = clean_str(df_dataK['text'])

In [6]:

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sentence_transformers import SentenceTransformer
def model_used(df, model):
    """Given a model choice, return the model and the computed matrix"""
    if model == 'Tfidf':
        tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 2), min_df=0)
        tfidf_matrix = tf.fit_transform(df['text'])
        return tf, tfidf_matrix
    elif model == 'CountVectorizer':
        cv = CountVectorizer(analyzer='word', ngram_range=(1, 2), min_df=0)
        matrix = cv.fit_transform(df['text'])
        return cv, matrix
    elif model == 'BERT':
#         bert = SentenceTransformer('distiluse-base-multilingual-cased-v1') # Multilingue
#         bert = SentenceTransformer('average_word_embeddings_glove.6B.300d') # + rapide
        bert = SentenceTransformer('paraphrase-MiniLM-L6-v2') # Meilleur score en théorie, à vérifier sur nos données
        matrix = bert.encode(df['text'].astype('str'), show_progress_bar=True)
        return bert, matrix

In [7]:

from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.preprocessing import LabelEncoder

# Set X and y
enc, X = model_used(df_dataK,'BERT')

Labelenc = LabelEncoder()
df_dataK['emotion'] = Labelenc.fit_transform(df_dataK['emotion'])
y = df_dataK['emotion']

You try to use a model that was created with version 1.2.0, however, your version is 1.1.1. This might cause unexpected behavior or errors. In that case, try to update to the latest version.



Batches: 100%|██████████| 1489/1489 [03:22<00:00,  7.36it/s]


In [8]:
# Prediction
#userinput= str(input()) 
userinput = "i love you so much!!!!"
print (userinput)
x = enc.encode(clean_str(pd.Series(lemmatizer(userinput))).values)

i love you so much!!!!


In [13]:
from reg_lin import get_metrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
params_LogisticRegression={
    "criterion":    ['gini', 'entropy'],    
    "n_estimators":[10,20,50,100,200,500,1000],
    "min_samples_split":[2,4,8,16],
    "min_samples_leaf":[2,4,8,16]}
    
model = RandomForestClassifier(n_jobs=-1)
truc = RandomizedSearchCV(model, param_distributions=params_LogisticRegression, scoring="accuracy",cv=3,verbose=3)

get_metrix(y,X,truc)

truc.best_params_

Fitting 3 folds for each of 10 candidates, totalling 30 fits
[CV] n_estimators=10, min_samples_split=4, min_samples_leaf=16, criterion=entropy 
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[CV]  n_estimators=10, min_samples_split=4, min_samples_leaf=16, criterion=entropy, score=0.441, total=   5.0s
[CV] n_estimators=10, min_samples_split=4, min_samples_leaf=16, criterion=entropy 
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    5.0s remaining:    0.0s
[CV]  n_estimators=10, min_samples_split=4, min_samples_leaf=16, criterion=entropy, score=0.438, total=   3.6s
[CV] n_estimators=10, min_samples_split=4, min_samples_leaf=16, criterion=entropy 
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    8.6s remaining:    0.0s
[CV]  n_estimators=10, min_samples_split=4, min_samples_leaf=16, criterion=entropy, score=0.439, total=   3.4s
[CV] n_estimators=100, min_samples_split=2, min_samples_leaf=8, criterion=gini 
[CV]  n_estimators=100, min_samples_

{'n_estimators': 1000,
 'min_samples_split': 8,
 'min_samples_leaf': 2,
 'criterion': 'entropy'}

In [14]:
truc.best_score_

0.48179933338582265

In [2]:
from reg_lin import get_metrix
from sklearn.linear_model import LogisticRegression ,SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from keras.models import Sequential 
from tensorflow.keras import layers, preprocessing
from tensorflow import keras

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score,accuracy_score


models = [  LogisticRegression(n_jobs=-1,multi_class='multinomial',penalty='l2',solver = 'saga' , max_iter=500,c=1 ),
            SGDClassifier(n_jobs=-1,penalty='l2', max_iter=500,loss='hinge',alpha=0.001),
            RandomForestClassifier(n_jobs=-1,n_estimators= 1000,min_samples_split= 8,min_samples_leaf= 2,criterion= 'entropy'),
            XGBClassifier(),
            Sequential())

place = 4
savedmodel=[]
count = 0
for modelset in models :
    if count == place:
        
        df_dataKKeras = df_dataK . sample (15000)
        y = df_dataKKeras['emotion']

        df_dataKKeras = df_dataKKeras[['text']]
        tk = preprocessing.text.Tokenizer()
        tk.fit_on_texts(df_dataKKeras['text'])
        X = tk.texts_to_matrix(df_dataKKeras['text'], mode='tfidf')
        
        x = tk.texts_to_matrix([userinput], mode='tfidf')
    
        X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=None)
        model = modelset
        model.add(layers.Dense(160, activation = 'relu', input_shape= [X_train.shape[1]]))
        model.add(layers.Dense(80, activation = 'relu'))
        model.add(layers.Dense(40, activation='relu'))
        model.add(layers.Dense(6,activation='softmax'))
        model.compile(optimizer="adam",loss="binary_crossentropy",metrics=['accuracy']) 
        model.fit(X_train,y_train,epochs=10, batch_size=100)

        y_train_pred = model.predict_classes(X_train)
        y_test_pred = model.predict_classes(X_test)
        
        print (model)

        print (" Train accuracy = {}" 
        .format(round(accuracy_score(y_train, y_train_pred),3)))

        print (" Test accuracy = {}" 
        .format(round(accuracy_score(y_test, y_test_pred),3)))
    if count != place:
        model = get_metrix(y,X,modelset)
    pred = model.predict(x)
    print (pred)
    if count == place:
        pred = Labelenc.inverse_transform([round(pred[0][0])])
    if count != place:
        pred = Labelenc.inverse_transform(pred)
    print (pred)
    savedmodel.append(model)
    count += 1
    print("\n")

SyntaxError: closing parenthesis ')' does not match opening parenthesis '{' on line 12 (<ipython-input-2-f47ce9092c25>, line 14)

In [9]:
del x,X,y,X_test,X_train,y_test,y_train,y_test_pred,y_train_pred,df_dataK,df_dataKKeras

In [1]:
# Import data
df_dataK = pd.read_csv(r'../data/d03_cleaned_data/datall_test.csv')

df_dataK['text'] = df_dataK['text'].apply(lemmatizer)
df_dataK['text'] = clean_str(df_dataK['text'])
X_test = enc.encode(df_dataK['text'])
y_test = Labelenc.transform(df_dataK['emotion'])

count = 0
place = 4 

for model in savedmodel:

    print (model)
    if count != place:
        y_test_pred = model.predict(X_test)
    if count == place:
        X_test = tk.texts_to_matrix(df_dataK['text'], mode='tfidf')
        y_test_pred = model.predict(X_test)
    print (" Test accuracy = {}" 
        .format(round(accuracy_score(y_test, y_test_pred),3)))

    count += 1


NameError: name 'pd' is not defined